# Ingeniería de Prompts  & Prototipado del Agente

#### **Objetivo**
Desarrollar y prototipar el agente mediante el diseño de prompts efectivos y su integración en un prototipo funcional. Este notebook combina la ingeniería de prompts con el prototipado del agente, permitiendo iteraciones y pruebas rápidas.

#### **Secciones**

1. **Ingeniería de Prompts**
   - Diseñar y probar diversas estructuras de prompts para asegurar que el modelo realice las tareas deseadas.
   - Crear prompts específicos para cada responsabilidad del agente.
   - Evaluar la efectividad de los prompts a través de pruebas iniciales.

2. **Prototipado del Agente**
   - Implementar la estructura básica del agente e integrar los prompts diseñados.
   - Desarrollar funciones esenciales (ej., llamadas a APIs, manejo de entradas).
   - Ejecutar pruebas iniciales y validar el rendimiento del agente.

3. **Iteración y Refinamiento**
   - Ajustar los prompts y funciones del agente según los resultados de las pruebas.
   - Refinar el agente para mejorar su desempeño.



### Prototipado del Nodo `classify_tasks` y su Prompt

En esta etapa se realiza el prototipado del primer componente del agente: el nodo `classify_tasks`. Este nodo es responsable de interpretar la entrada del usuario y clasificar qué tareas están presentes (clima, divisas, noticias). Además del código funcional, se prueba y valida el prompting específico que permitirá al modelo ejecutar esta clasificación de forma consistente.

---

#### 🎯 Objetivo de la prueba

Validar el correcto funcionamiento del nodo `classify_tasks`, incluyendo:

- El diseño y comportamiento del prompt que guía al LLM para identificar tareas dentro de un `user_input`.
- El manejo de errores en la llamada al modelo.
- La consistencia en la estructura de la respuesta.

---

#### 🧠 Lógica del Nodo

`classify_tasks` debe:

- Recibir una entrada de usuario (`user_input`).
- Usar un prompt diseñado para pedir al LLM que identifique si el texto menciona temas de:
  - Clima
  - Tipo de cambio
  - Noticias
- Devolver un diccionario con:
  ```python
  {
      "tasks": {
          "weather": True/False,
          "currency": True/False,
          "news": True/False
      },
      "error": None o mensaje de error
  }
  ```

En caso de error (por ejemplo, problemas con el LLM), el campo `"error"` debe incluir una descripción del fallo, y `"tasks"` tendrá todos los valores en `False`.

---

#### Componentes de la prueba

1. **Prompting**  
   Validar que el prompt sea claro, específico y conduzca al LLM a generar una salida estructurada y predecible. Un ejemplo de prompt podría ser:

   ```
   Dada la siguiente entrada del usuario, identifica si contiene menciones sobre:

   - Clima
   - Tipo de cambio (divisas)
   - Noticias generales

   Devuelve el resultado en formato JSON como este:
   {
     "tasks_to_do": {
       "weather": true/false,
       "currency": true/false,
       "news": true/false
     }
   }

   Entrada: <<user_input>>
   ```

2. **Pruebas funcionales del nodo**
   - Entradas con menciones claras de cada tipo de tarea.
   - Entradas mixtas (clima + noticias, divisas + clima, etc.).
   - Entradas ambiguas o vacías.
   - Casos de error simulados (por ejemplo, lanzar una excepción desde el LLM).

---

#### Pasos para ejecutar la prueba

1. **Configura el entorno**  
   - Asegúrate de tener instalada tu librería LLM (por ejemplo, `openai`) y tener una clave API válida.
   - Carga o define tu función `classify_tasks`.

In [1]:
# ir al directorio principal
from os import chdir

chdir("../")

In [2]:
# cargar varibles
from dotenv import load_dotenv
load_dotenv(dotenv_path='env')

from utils.logging import setup_logging

# Initialize logger using the setup_logging function
logger = setup_logging()

### Instala las bibliotecas necesarias

Ejecuta el siguiente comando en la terminal:

```bash
pip install -r requirements.txt


In [3]:
from langchain_openai import ChatOpenAI

# ----- Modelo -----
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

### Definir el `state` que compartirán los nodos y agentes del grafo

Este `state` es el contenedor central de información que se irá actualizando y transmitiendo entre nodos a lo largo del flujo definido en el grafo de LangGraph.


In [4]:
# %%writefile core/agent_state.py

from dotenv import load_dotenv
from typing import Annotated, TypedDict, Optional, List, Dict, Any
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

# Cargar variables de entorno
load_dotenv(dotenv_path='env')

# Función para combinar diccionarios
def merge_dicts(dict1, dict2):
    if not isinstance(dict1, dict) or not isinstance(dict2, dict):
        raise TypeError(f"Ambos argumentos deben ser diccionarios. Recibido: {type(dict1)} y {type(dict2)}")
    return {**dict1, **dict2}

def add_history_update(history_old: List[str], history_new: List[str]) -> List[str]:
    return history_old + history_new

# Clase AgentState
class AgentState(TypedDict):
    """
    Representa el estado que fluye a través del agente LangGraph.

    Atributos:
        messages (list[BaseMessage]):
            Lista de mensajes intercambiados entre el sistema y el usuario,
            gestionados automáticamente con el paso de mensajes de LangGraph.
        
        order_task (Optional[List[str]]):
            Lista opcional que representa el orden en el que las tareas identificadas 
            deben ser procesadas o presentadas.

        results (Dict[str, Any]):
            Diccionario que contiene los resultados de varios nodos de tareas.
            Cada clave es el nombre de la tarea, y el valor es el resultado o error.

        error (Optional[str]):
            Mensaje de error general para representar cualquier problema en el flujo del agente.

        tasks_to_do (Dict[str, bool]):
            Diccionario que representa qué tareas han sido identificadas para ejecutar.
            Ejemplo: {"weather": True, "currencies": True, "news": False}

        ready_to_aggregate (bool):
            Indica si todas las tareas esperadas están listas y el paso final de agregación puede ejecutarse.

        history (List[str]):
            Lista para realizar un seguimiento de los nombres de los nodos por los que pasa el flujo.
    """
    
    messages: Annotated[List[BaseMessage], add_messages]  # Mensajes intercambiados
    order_task: Dict[str, Any]  # Orden de las tareas
    error: Annotated[Dict[str, str], merge_dicts]  # Manejo de errores
    results: Annotated[Dict[str, str], merge_dicts]  # Resultados de las tareas
    task_completed: Annotated[Dict[str, str], merge_dicts]  # Tareas completadas
    tasks_to_do: Dict[str, bool]  # Tareas pendientes
    ready_to_aggregate: bool  # Indicador de si está listo para agregarse
    history: Annotated[List[str], add_history_update]  # Historial de nodos procesados



In [5]:
# %%writefile nodes/classify_query.py
from dotenv import load_dotenv

import logging
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from typing import cast
from core.agent_state import AgentState
import json

from utils.logging import setup_logging

# Initialize logger using the setup_logging function
logger = setup_logging()

# ----- Cargar variables de entorno -----
load_dotenv(dotenv_path='env')

# Global LLM instance
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


# ----- System message (prompt) -----
system_prompt = SystemMessage(
    content="""
Eres un asistente que clasifica tareas en tres categorías: weather, exchange y news.
Dado un mensaje del usuario, responde con un JSON con claves: "weather", "exchange", "news",
y valores booleanos indicando si la tarea está presente.
Ejemplo de respuesta: {"weather": true, "exchange": false, "news": true}
"""
)

# ----- Node: classify_tasks -----
def classify_tasks(state: AgentState) -> AgentState:
    """
    Clasifica la intención del usuario en categorías predefinidas
    usando un modelo de lenguaje y actualiza el estado.
    """
    # Añadir trazabilidad del nodo
    state.setdefault("history", []).append("classify_tasks")
    try:
        logger.debug("Buscando el último mensaje del usuario...")
        user_msg = [m for m in state["messages"] if isinstance(m, HumanMessage)][-1]
        logger.info(f"Mensaje recibido: {user_msg.content}")

        full_prompt = [system_prompt, user_msg]

        logger.debug("Enviando prompt al modelo...")
        response = llm.invoke(full_prompt)
        logger.debug(f"Respuesta del modelo: {response.content}")

        classification = json.loads(response.content)
        classification = cast(dict, classification)
        logger.info(f"Tareas clasificadas: {classification}")

        new_state = {
            "tasks_to_do": classification,
            "results": state.get("results", {}),
            "task_completed": {},
            "error": {},
            "order_task": {},
            "ready_to_aggregate": False,
        }

        logger.debug("Estado actualizado correctamente.")
        return new_state

    except Exception as e:
        error_msg = f"classify: {str(e)}"
        logger.exception("Error al clasificar el mensaje del usuario.")
        return {
            "messages": state["messages"] + [SystemMessage(content=error_msg)],
            "results": state.get("results", {}),
            "tasks_to_do": {},
            "error": {"classify": error_msg},
            "order_task": {},
            "ready_to_aggregate": False,
            "task_completed": {}
        }


2. **Ejecuta ejemplos de prueba**

 Define diferentes entradas de usuario para probar varios escenarios:

In [6]:
test_inputs = [
    # Casos con claridad
    "¿Cómo está el clima en Ciudad de México?",  # Clima
    "¿Cuál es el tipo de cambio de USD a EUR hoy?",  # Divisas
    "Dame las últimas noticias de México.",  # Noticias
    # Casos combinados
    "Quiero saber el clima y las noticias.",  # Clima, Noticias
    "Tipo de cambio del dólar.",  # Divisas
    "Noticias del fútbol y el clima mañana.",  # Clima, Noticias
    # Casos ambiguos o sin relación
    "Algo completamente sin relación.",  # Ninguno
    "No entiendo qué pasa con el clima",  # Clima (con ambigüedad)
    "El dólar está cayendo, pero no sé sobre el clima",  # Divisas
    "¿Hay noticias del fútbol?",  # Noticias
    # Casos de error
    "",  # Entrada vacía
    "    ",  # Entrada vacía con espacios
    "¿Cómo va el clima pero no sé si te refieres a Ciudad de México?"  # Clima, ambigüedad
]


In [7]:
# ----- Ejemplo de prueba de llamada -----
for input_text in test_inputs:
    state = {
        "messages": [HumanMessage(content=input_text)],
    }
    
    output = classify_tasks(state)
    
    print(f"Entrada: '{input_text}'")
    # Verificamos si el mensaje final tiene un contenido válido
    print(f"Salida: {output}")
    print("-" * 40)

Entrada: '¿Cómo está el clima en Ciudad de México?'
Salida: {'tasks_to_do': {'weather': True, 'exchange': False, 'news': False}, 'results': {}, 'task_completed': {}, 'error': {}, 'order_task': {}, 'ready_to_aggregate': False}
----------------------------------------
Entrada: '¿Cuál es el tipo de cambio de USD a EUR hoy?'
Salida: {'tasks_to_do': {'weather': False, 'exchange': True, 'news': False}, 'results': {}, 'task_completed': {}, 'error': {}, 'order_task': {}, 'ready_to_aggregate': False}
----------------------------------------
Entrada: 'Dame las últimas noticias de México.'
Salida: {'tasks_to_do': {'weather': False, 'exchange': False, 'news': True}, 'results': {}, 'task_completed': {}, 'error': {}, 'order_task': {}, 'ready_to_aggregate': False}
----------------------------------------
Entrada: 'Quiero saber el clima y las noticias.'
Salida: {'tasks_to_do': {'weather': True, 'exchange': False, 'news': True}, 'results': {}, 'task_completed': {}, 'error': {}, 'order_task': {}, 'ready

In [8]:
### ver salida completa 

In [9]:
output

{'tasks_to_do': {'weather': True, 'exchange': False, 'news': False},
 'results': {},
 'task_completed': {},
 'error': {},
 'order_task': {},
 'ready_to_aggregate': False}

3. **Evalúa resultados**
   - ¿El modelo identificó correctamente las tareas?
   - ¿La salida sigue el formato esperado?
   - ¿Hay algún comportamiento inesperado?



### 🔍 Análisis rápido del caso

**Entrada:** "El dólar está cayendo, pero no sé sobre el clima"

**Salida:**  
```json
{
  "weather": false,
  "divisas": true,
  "noticias": true
}
```

**Problema:**  
El modelo activó `noticias` simplemente por la mención de una *situación económica*, aunque no se haya pedido explícitamente noticias. Esto revela que está interpretando más allá del *task intent*, y no tanto como una consulta.

---

### 💡 Estrategias de Prompting para Mejorar Clasificación



#### 1. **Prompt con desambiguación explícita por intención**
En lugar de sólo pedir que clasifique temas, puedes instruir al modelo a buscar **intenciones explícitas de consulta**, no solo menciones.

```text
Clasifica si el usuario QUIERE información sobre clima, divisas o noticias. 
No respondas True si solo se menciona el tema, sino si hay una intención clara de solicitarlo.
```

Esto ayuda a evitar que se activen `True` por frases como “no sé sobre el clima”, ya que no está pidiendo información, solo haciendo referencia.

Va, aquí va un resumen más breve y directo, con las estrategias bien concentradas:

2. **Few-shot con ejemplos ambiguos**  
   Dar ejemplos donde hay mención sin petición para que el modelo aprenda a distinguir.

3. **Preprocesamiento o reformulación por un agente previo**  
   Un agente intermedio puede reestructurar el mensaje del usuario para alinear mejor con el objetivo de clasificación (quitando ruido, reordenando, o aclarando ambigüedades).

4. **Edición del prompt para filtrar partes irrelevantes**  
   Por ejemplo, eliminar frases como “el dólar está cayendo” si no implican una petición directa, o convertir "pero no sé sobre el clima" en "quiero saber el clima".



### 🔍 Análisis rápido del caso

**Entrada:** "El dólar está cayendo, pero no sé sobre el clima"

**Salida:**  
```json
{
  "weather": false,
  "divisas": true,
  "noticias": true
}
```

**Problema:**  
El modelo activó `noticias` simplemente por la mención de una *situación económica*, aunque no se haya pedido explícitamente noticias. Esto revela que está interpretando más allá del *task intent*, y no tanto como una consulta.

---

### 💡 Estrategias de Prompting para Mejorar Clasificación



#### 1. **Prompt con desambiguación explícita por intención**
En lugar de sólo pedir que clasifique temas, puedes instruir al modelo a buscar **intenciones explícitas de consulta**, no solo menciones.

```text
Clasifica si el usuario QUIERE información sobre clima, divisas o noticias. 
No respondas True si solo se menciona el tema, sino si hay una intención clara de solicitarlo.
```

Esto ayuda a evitar que se activen `True` por frases como “no sé sobre el clima”, ya que no está pidiendo información, solo haciendo referencia.

Va, aquí va un resumen más breve y directo, con las estrategias bien concentradas:

2. **Few-shot con ejemplos ambiguos**  
   Dar ejemplos donde hay mención sin petición para que el modelo aprenda a distinguir.

3. **Preprocesamiento o reformulación por un agente previo**  
   Un agente intermedio puede reestructurar el mensaje del usuario para alinear mejor con el objetivo de clasificación (quitando ruido, reordenando, o aclarando ambigüedades).

4. **Edición del prompt para filtrar partes irrelevantes**  
   Por ejemplo, eliminar frases como “el dólar está cayendo” si no implican una petición directa, o convertir "pero no sé sobre el clima" en "quiero saber el clima".



### Prototipado del Nodo "Clima" y su Prompt

En esta etapa se realiza el prototipado del nodo **get_weather**, que tiene como objetivo extraer el nombre de la ciudad de la entrada del usuario y consultar la API de OpenWeather para obtener el clima de dicha ciudad. Además, se prueba y valida el diseño del **prompt** que guía al modelo LLM para extraer correctamente el nombre de la ciudad.

🎯 **Objetivo de la prueba**  
Validar el funcionamiento del nodo **get_weather**, asegurando que:

- El prompt sea efectivo para extraer el nombre correcto de la ciudad desde un texto de entrada del usuario.
- El nodo maneje correctamente los errores como entradas mal formadas, ciudades no reconocidas o problemas con la API de OpenWeather.
- La respuesta tenga la estructura esperada con la información del clima o un mensaje de error adecuado.

🧠 **Lógica del Nodo**  
**get_weather** debe:

1. Recibir una entrada de usuario con un mensaje de texto (por ejemplo, "¿Cómo está el clima en Nueva York?").
2. Usar un **prompt** para extraer el nombre de la ciudad en inglés de dicho texto.
3. Consultar la API de OpenWeather con el nombre de la ciudad extraída.
4. Devolver un diccionario con:
   ```python
   {
       "results": ["Descripción del clima"],
       "error": None o mensaje de error
   }
   ```
   En caso de error (por ejemplo, problemas con la API o ciudad no encontrada), el campo "error" debe contener una descripción detallada, y "results" estará vacío.


In [10]:
# %%writefile agents/weather_agent.py

from dotenv import load_dotenv
import os
import requests
import logging
from typing import Optional
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph.message import add_messages
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

from utils.logging import setup_logging

# Initialize logger using the setup_logging function
logger = setup_logging()

# ----- Load environment variables -----
load_dotenv(dotenv_path='env')

# Global LLM instance
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# ----- Prompt Template -----
city_extraction_template = """
Eres un asistente que extrae el nombre de la ciudad en inglés americano del siguiente texto. 
Responde solo con el nombre de la ciudad, sin comillas ni símbolos extra.

Ejemplo:
Texto: "¿Cómo está el clima en Nueva York?" -> New York
Texto: "{text}"
"""

city_extraction_prompt = PromptTemplate(input_variables=["text"], template=city_extraction_template)

# ----- LLM-based city extractor -----
def extract_city_with_llm(text: str) -> Optional[str]:
    """
    Extracts the name of the city from the given text using a language model.

    Args:
    - text (str): The input text that may contain a city name.

    Returns:
    - str or None: Returns the city name if successfully extracted, otherwise None.
    """
    logger.debug(f"Extracting city from text: '{text}'")
    prompt = city_extraction_prompt.format(text=text)

    try:
        response = llm.invoke([HumanMessage(content=prompt)])
        city = response.content.strip()
        logger.info(f"City extracted: '{city}'")
    except Exception as e:
        logger.exception("Error invoking the model for city extraction.")
        return None

    if not city or len(city) < 2 or any(c in city for c in ['{', '}', '[', ']']):
        logger.warning(f"Invalid city detected: '{city}'")
        return None

    return city

# ----- Weather Node -----
def get_weather(state: AgentState) -> AgentState:
    """
    Handles weather-related queries using an LLM to extract the city and the OpenWeatherMap API to fetch weather data.

    Returns:
    - 'results': If successful, a dictionary with the weather report.
    - 'error': If an issue occurs, a dictionary with the error message.
    - 'task_completed': A boolean flag indicating if the task was completed.
    """
    # Añadir trazabilidad del nodo
    state.setdefault("history", []).append("task_weather")

    try:
        input_text = state["messages"][-1].content
        logger.debug(f"Received weather message: '{input_text}'")

        city = extract_city_with_llm(input_text)
        logger.debug(f"Respose llm: '{city}'")

        if not city:
            msg = "City could not be identified in the message."
            logger.warning(msg)
            return {
                "error": {"weather": msg},
                "task_completed": {"weather": False}
            }

        api_key = os.getenv("OPENWEATHER_API_KEY")
        if not api_key:
            msg = "API Key not configured in the system."
            logger.error(msg)
            return {
                "error": {"weather": msg},
                "task_completed": {"weather": False}
            }

        logger.info(f"Fetching weather for: {city}")
        location_url = "https://api.openweathermap.org/data/2.5/weather"
        params = {
            "q": city,
            "appid": api_key,
            "units": "metric"
        }
        location_response = requests.get(location_url, params=params)

        if location_response.status_code != 200:
            msg = f"City '{city}' not found or not correctly written in English."
            logger.warning(msg)
            return {
                "error": {"weather": msg},
                "task_completed": {"weather": False}
            }

        location_data = location_response.json()

        try:
            weather_desc = location_data["weather"][0]["description"]
            temperature = location_data["main"]["temp"]
        except (KeyError, IndexError, TypeError) as e:
            msg = "Unexpected weather data format received from API."
            logger.exception(msg)
            return {
                "error": {"weather": msg},
                "task_completed": {"weather": False}
            }

        weather_report = f"The weather in {city} is {weather_desc} with a temperature of {temperature}°C."
        logger.info(f"Generated weather report: {weather_report}")

        return {
            "results": {"weather": [weather_report]},
            "task_completed": {"weather": True}
        }

    except Exception as e:
        msg = f"Error obtaining weather: {str(e)}"
        logger.exception(msg)
        return {
            "error": {"weather": msg},
            "task_completed": {"weather": False}
        }


In [11]:
# ----- Ejemplo de prueba con distintos casos -----
test_inputs = [
    # Casos comunes
    "¿Cómo está el clima en Nueva York?",  # Ciudad conocida
    "¿Cuál es el clima en Los Ángeles?",  # Ciudad con nombre compuesto
    "Me gustaría saber cómo está el clima en Londres.",  # Ciudad con nombre en inglés
    "¿Qué tal el clima en Ciudad de México?",  # Ciudad con nombre compuesto en español

    # Casos especiales
    "¿Qué tal está el clima?",  # Sin mención de ciudad
    "El clima en 1234 podría cambiar.",  # Texto con números, no ciudad
    "¿Cómo está el clima en Paris#@",  # Ciudad con caracteres especiales
    "Hace calor en Barcelon",  # Ciudad con nombre parcialmente correcto
    "¿Cómo está el clima en Shanghai?",  # Ciudad conocida en otro idioma
    "¿En qué estado está el clima de Berlín?",  # Pregunta con "estado", no ciudad
    "¿Cuánto falta para llegar a Tokyo? Me olvidé de la hora.",  # Ciudad mencionada con información extra
    "¿Cómo está el clima en Saint Petersburg?",  # Ciudad compuesta con un 'Saint'
    "¿Cómo está el clima en Amsterdam?",  # Ciudad en inglés sin acento
]

In [12]:
# ----- Ejemplo de prueba con el ciclo -----
for input_text in test_inputs:
    state = {
                "messages": [HumanMessage(content=input_text)],
                "tasks_to_do": {},
                "results": {},
                "error": {},
                "order_task": None,
                "ready_to_aggregate": False,
            }
    # Llamada al nodo de clima
    output = get_weather(state)
    
    print(f"Entrada: '{input_text}'")
    print(f"Salida: {output}")


Entrada: '¿Cómo está el clima en Nueva York?'
Salida: {'results': {'weather': ['The weather in New York is clear sky with a temperature of 22.1°C.']}, 'task_completed': {'weather': True}}
Entrada: '¿Cuál es el clima en Los Ángeles?'
Salida: {'results': {'weather': ['The weather in Los Angeles is overcast clouds with a temperature of 15.93°C.']}, 'task_completed': {'weather': True}}
Entrada: 'Me gustaría saber cómo está el clima en Londres.'
Salida: {'results': {'weather': ['The weather in Londres is clear sky with a temperature of 9.96°C.']}, 'task_completed': {'weather': True}}
Entrada: '¿Qué tal el clima en Ciudad de México?'
Salida: {'results': {'weather': ['The weather in Mexico City is broken clouds with a temperature of 26.64°C.']}, 'task_completed': {'weather': True}}
Entrada: '¿Qué tal está el clima?'
Salida: {'error': {'weather': "City 'No city mentioned' not found or not correctly written in English."}, 'task_completed': {'weather': False}}
Entrada: 'El clima en 1234 podría c

In [13]:
output

{'results': {'weather': ['The weather in Amsterdam is few clouds with a temperature of 13.11°C.']},
 'task_completed': {'weather': True}}


### Consideraciones para el uso de LLMs para la extracción de ciudades

Aunque los LLMs son bastante efectivos para extraer información de textos, hay ciertos aspectos que pueden influir en la precisión de la extracción, especialmente cuando se trata de errores tipográficos o casos especiales. A continuación se presentan algunos puntos clave a considerar:

- **Errores tipográficos menores**: Los LLMs suelen manejar errores tipográficos leves (por ejemplo, "Nwe York" en lugar de "New York"), pero pueden fallar si los errores son más significativos (como "Nuw York" o "New Yrok").
  
- **Casos de ciudades homónimas**: En situaciones donde hay varias ciudades con el mismo nombre (como "Paris" en EE. UU. y "París" en Francia), un LLM puede no distinguir correctamente entre ellas, especialmente si el contexto es ambiguo.

- **Contexto ambiguo**: Si el contexto en el texto no es claro o hay múltiples interpretaciones posibles (por ejemplo, "La ciudad de México"), el LLM podría extraer una ciudad incorrecta o no estar seguro de qué ciudad se menciona.

- **Limitación en la precisión**: Aunque son robustos, los LLMs no siempre son infalibles para identificar la ciudad con precisión, especialmente cuando hay nombres de ciudades poco comunes o errores más complejos en el texto.

#### Posible mejora:
Se podría implementar un sistema **NERD (Reconocimiento de Entidades Nombradas)** especializado en ciudades para manejar estos casos con mayor precisión, ya que está diseñado para identificar entidades específicas de manera más confiable que los modelos de lenguaje general.

---

### Prototipado del Nodo "Divisas" y su Prompt

En esta etapa se realiza el prototipado del nodo **get_exchange_rate**, cuyo objetivo es obtener la tasa de cambio entre el dólar estadounidense (USD) y el euro (EUR) usando la API de ExchangeRate. Además, se valida el diseño del **prompt** que guiará al modelo para realizar esta consulta correctamente.

🎯 **Objetivo de la prueba**  
Validar el funcionamiento del nodo **get_exchange_rate**, asegurando que:

- El nodo realice la consulta correctamente a la API de ExchangeRate.
- El manejo de errores sea adecuado en caso de problemas con la API o de respuesta inesperada.
- El resultado tenga la estructura esperada, mostrando la tasa de cambio de USD a EUR.

🧠 **Lógica del Nodo**  
**get_exchange_rate** debe:

1. Consultar la API de ExchangeRate con la clave de API configurada.
2. Obtener la tasa de cambio entre USD y EUR.
3. Devolver un diccionario con:
   ```python
   {
       "results": ["1 USD = X EUR"],
       "error": None o mensaje de error
   }
   ```
   En caso de error (como problemas con la API), el campo "error" debe contener la descripción detallada, y "results" estará vacío.

---



In [14]:
# %%writefile agents/currency_agent.py

import os
import logging
import requests
from typing import Optional
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

# ----- Configure logging -----
from utils.logging import setup_logging

# Initialize logger using the setup_logging function
logger = setup_logging()

# ----- Load environment variables -----
# This loads environment variables from a .env file, typically containing sensitive information like API keys.
load_dotenv(dotenv_path='env')

# ----- Global LLM instance -----
# Initializes the language model for currency extraction. 
# The model used here is GPT-3.5-turbo, which is ideal for text processing and extraction tasks.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# ----- Prompt Template for currency extraction -----
# This is the template used to instruct the language model to extract currency codes (ISO 4217 format) from the given text.
# The prompt is written in Spanish but will be used to parse any input text in the same format.
currency_extraction_template = """
Eres un asistente que extrae dos códigos de divisas (ISO 4217) desde el texto dado. 
Responde únicamente con los códigos separados por coma. No uses símbolos ni explicaciones.

Ejemplo:
Texto: "¿Cuánto vale un dólar en pesos mexicanos?" -> USD, MXN
Texto: "{text}"
"""

currency_extraction_prompt = PromptTemplate(
    input_variables=["text"],  # The input variable that will be passed to the template is 'text'.
    template=currency_extraction_template  # The prompt template for extraction.
)

# ----- Function to extract currencies using the language model (LLM) -----
def extract_currencies_with_llm(text: str) -> Optional[tuple[str, str]]:
    """
    Extracts two currency codes from a given text using the pre-defined language model prompt.
    
    Parameters:
    text (str): The input text containing the currencies to be extracted.
    
    Returns:
    Optional[tuple[str, str]]: A tuple containing two ISO 4217 currency codes (or None if extraction fails).
    """
    try:
        # Format the prompt with the provided text
        prompt = currency_extraction_prompt.format(text=text)
        logger.debug(f"Prompt sent to LLM: {prompt}")
        
        # Get the response from the LLM
        response = llm.invoke([HumanMessage(content=prompt)])
        result = response.content.strip()
        logger.debug(f"LLM response: {result}")

        # Split the result into two parts (currency codes)
        parts = [p.strip().upper() for p in result.split(",")]

        # Check if the result contains exactly two 3-letter currency codes
        if len(parts) == 2 and all(len(code) == 3 for code in parts):
            logger.info(f"Successfully extracted currency codes: {parts}")
            return parts[0], parts[1]
        else:
            logger.warning(f"Unexpected format in the response: {result}")

    except Exception as e:
        # Log the error if extraction fails
        logger.exception("Error during currency extraction")

    return None

# ----- Function to get exchange rate -----
def get_exchange_rate(state: AgentState) -> AgentState:
    """
    Processes the user's message to detect currencies and fetches the exchange rate between them.
    
    Parameters:
    state (dict): The current state of the agent, which contains the user's message and other context.
    
    Returns:
    dict: The updated state dictionary containing either the results or error messages.
    """
    # Añadir trazabilidad del nodo
    state.setdefault("history", []).append("task_exchange")
    try:
        # Get the input text from the state (the user's message)
        input_text = state["messages"][-1].content
        logger.info(f"Processing user message: {input_text}")

        # Extract the currency codes from the user's input
        currencies = extract_currencies_with_llm(input_text)
        
        # If no currencies are detected, return an error
        if not currencies:
            msg = "No currencies detected in the message."
            logger.warning(msg)
            return {
                "error": {"exchange": msg},
                "task_completed":{"exchange": True} 
            }

        # Extract base and target currencies
        base_currency, target_currency = currencies
        logger.info(f"Detected currencies: {base_currency} -> {target_currency}")

        # Retrieve the API key for the exchange rate service from environment variables
        api_key = os.getenv("EXCHANGE_API_KEY")
        if not api_key:
            logger.error("API Key not set in the environment.")
            return {
                "error": {"exchange": "API Key not configured in the system."},
                "task_completed":{"exchange": True} 
            }

        # Construct the API URL to get the exchange rates
        url = f"https://v6.exchangerate-api.com/v6/{api_key}/latest/{base_currency}"
        logger.debug(f"Querying external API: {url}")
        response = requests.get(url)

        # Check if the response from the API is successful
        if response.status_code != 200:
            logger.error(f"Error in API response: {response.status_code}")
            return {
                "error": {"exchange": f"API error: {response.status_code}"},
                "task_completed":{"exchange": False} 
            }

        # Parse the JSON data from the API response
        data = response.json()

        # Check if the target currency's exchange rate is available
        if target_currency not in data.get('conversion_rates', {}):
            logger.warning(f"Exchange rate for {target_currency} not available in the response.")
            return {
                "error": {"exchange": f"Exchange rate for {target_currency} not found."},
                "task_completed":{"exchange": True} 
            }

        # Get the exchange rate for the target currency and format the response message
        rate = data['conversion_rates'][target_currency]
        message = f"1 {base_currency} = {rate} {target_currency}"
        logger.info(f"Exchange rate obtained: {message}")

        # Return the exchange rate result in the updated state
        return {
            "results": {"exchange": [message]},
            "task_completed":{"exchange": True} 
        }

    except Exception as e:
        # Handle any unexpected errors and return them in the state
        logger.exception("Unexpected error while getting exchange rate")
        return {
            "error": {"exchange": f"Error obtaining exchange rate: {str(e)}"},
            "task_completed":{"exchange": False} 
        }


In [15]:
# ----- Ejemplo de prueba con el ciclo -----
test_inputs = [
    "Quiero saber la tasa de cambio de Estados Unidos y también el clima de Nueva York.",
    "¿Cuál es la tasa de cambio de euros y cómo está el clima en Londres?",
    "Me interesa saber cuánto está el dólar en euros y qué tiempo hace en París.",
    "Quiero la tasa de cambio entre el dólar y el euro, además del clima en Tokio.",
    "¿Cómo está el clima en Madrid y qué tal la tasa de cambio del dólar?",
    "Dime la tasa de cambio de Europa y el clima de Los Ángeles.",
    "Quiero saber la tasa de cambio de Europa y cómo está el clima en Barcelona.",
    "¿Me dices la tasa de cambio del dólar a euro y también qué clima hace en Buenos Aires?",
    "¿Cuánto está el euro en dólares y cómo está el clima en Ciudad de México?",
    "Estoy buscando la tasa de cambio entre el dólar y el euro, además del clima de Berlín."
]

for input_text in test_inputs:
    state = {
                "messages": [HumanMessage(content=input_text)],
                "tasks_to_do": {},
                "results": {},
                "error": {},
                "order_task": None,
                "ready_to_aggregate": False,
            }

    output = get_exchange_rate(state)

    print(f"🧪 Entrada: '{input_text}'")
    print(f"🧪 output: '{output}'")


🧪 Entrada: 'Quiero saber la tasa de cambio de Estados Unidos y también el clima de Nueva York.'
🧪 output: '{'results': {'exchange': ['1 USD = 19.6192 MXN']}, 'task_completed': {'exchange': True}}'
🧪 Entrada: '¿Cuál es la tasa de cambio de euros y cómo está el clima en Londres?'
🧪 output: '{'results': {'exchange': ['1 EUR = 0.8567 GBP']}, 'task_completed': {'exchange': True}}'
🧪 Entrada: 'Me interesa saber cuánto está el dólar en euros y qué tiempo hace en París.'
🧪 output: '{'results': {'exchange': ['1 USD = 0.8764 EUR']}, 'task_completed': {'exchange': True}}'
🧪 Entrada: 'Quiero la tasa de cambio entre el dólar y el euro, además del clima en Tokio.'
🧪 output: '{'results': {'exchange': ['1 USD = 0.8764 EUR']}, 'task_completed': {'exchange': True}}'
🧪 Entrada: '¿Cómo está el clima en Madrid y qué tal la tasa de cambio del dólar?'
🧪 output: '{'results': {'exchange': ['1 USD = 0.8764 EUR']}, 'task_completed': {'exchange': True}}'
🧪 Entrada: 'Dime la tasa de cambio de Europa y el clima de 

--- 

### Prototipado del Nodo "Noticias" y su Prompt

En esta etapa se realiza el prototipado del nodo **get_news**, cuyo objetivo es obtener los titulares más importantes actuales de un país usando la API de NewsAPI. Se valida el diseño del **prompt** que guiará al modelo para realizar la consulta y mostrar los titulares.

🎯 **Objetivo de la prueba**  
Validar el funcionamiento del nodo **get_news**, asegurando que:

- El nodo realice correctamente la consulta a la API de NewsAPI.
- El manejo de errores sea adecuado en caso de problemas con la API o respuesta inesperada.
- El resultado tenga la estructura esperada, mostrando los titulares de las noticias más recientes.

🧠 **Lógica del Nodo**  
**get_news** debe:

1. Consultar la API de NewsAPI con la clave de API configurada.
2. Obtener los titulares de noticias en algún pais.
3. Devolver un diccionario con:
   ```python
   {
       "results": ["Titular 1, Titular 2, Titular 3"],
       "error": None o mensaje de error
   }
   ```
   En caso de error (como problemas con la API), el campo "error" debe contener la descripción detallada, y "results" estará vacío.

In [16]:
# %%writefile agents/news_agent.py

import os
import logging
import requests
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

# ----- Configure logging -----
from utils.logging import setup_logging

# Initialize logger using the setup_logging function
logger = setup_logging()

# ----- Load environment variables -----
# Loads environment variables from the .env file, typically containing API keys like the News API key.
load_dotenv(dotenv_path='env')

# ----- Global LLM instance -----
# This initializes the LLM (language model) from OpenAI with the "gpt-3.5-turbo" model.
# It's used for natural language understanding and extracting country codes from text.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# ----- Prompt Template for country extraction -----
# This is the prompt used by the LLM to extract the country code (ISO 3166-1 alpha-2) from the provided text.
# The prompt asks for a 2-letter country code and returns only that code.
country_extraction_template = """
You are an assistant that extracts the country (in ISO 3166-1 alpha-2 code, like 'MX', 'US', 'FR') from the following text.
Respond only with the country code. If no country is mentioned, respond with ' '.

Text: "{text}"
"""

# Create a prompt template that the LLM will use.
country_extraction_prompt = PromptTemplate(
    input_variables=["text"],  # The input variable for the template is 'text'.
    template=country_extraction_template  # The actual template for extraction.
)

# ----- Function to extract country from the text using the LLM -----
def extract_country_with_llm(text: str) -> str:
    """
    Extracts the country code from the provided text using the LLM.

    Parameters:
    text (str): The input text containing a country mention.

    Returns:
    str: The ISO 3166-1 alpha-2 country code extracted from the text (or ' ' if no country is mentioned).
    """
    try:
        # Format the prompt with the provided text
        prompt = country_extraction_prompt.format(text=text)
        logger.debug(f"Prompt sent to LLM: {prompt}")
        
        # Get the response from the LLM
        response = llm.invoke([HumanMessage(content=prompt)])
        country = response.content.strip().lower()  # Normalize the country code (convert to lowercase)
        logger.debug(f"LLM response: {country}")

        # Validate that the response is a valid 2-letter country code
        if len(country) != 2 or not country.isalpha():
            logger.warning("Invalid response from LLM")
            return None  # Return a blank space if the response is invalid
        return country

    except Exception as e:
        # Log any exception that occurs during country extraction
        logger.exception("Error during country extraction")
        return None  # Return 'None' as a default fallback

# ----- News fetching function -----
def get_news(state: AgentState) -> AgentState:
    """
    Processes the input text to detect the country and fetches news headlines for that country.

    Parameters:
    input_text (str): The input text containing user query or message.

    Returns:
    dict: A dictionary containing the news headlines or error information.
    """
    # Añadir trazabilidad del nodo
    state.setdefault("history", []).append("task_news")
    
    # Get the input text from the state (the user's message)
    input_text = state["messages"][-1].content
    try:
        logger.info(f"Processing user message: {input_text}")

        # Extract the country code from the user's message
        country_code = extract_country_with_llm(input_text)
        logger.info(f"Detected country code: {country_code}")

        # Retrieve the News API key from the environment variables
        api_key = os.getenv("NEWS_API_KEY")
        if not api_key:
            msg = "News API Key is not configured."
            logger.error(msg)
            return {
                    "error": {"news": msg},
                   "task_completed": {"news": False}
            }

        # Construct the API URL to get news headlines for the detected country
        url = f"https://newsapi.org/v2/top-headlines?country={country_code}&apiKey={api_key}"
        logger.debug(f"Querying News API: {url}")
        response = requests.get(url)

        # Handle non-200 HTTP responses from the News API
        if response.status_code != 200:
            msg = f"Error in News API: {response.status_code}"
            logger.error(msg)
            return {
                "error": {"news": msg},
                "task_completed": {"news": False}
            }

        # Parse the JSON response from the News API
        data = response.json()

        # Check if the 'articles' key exists in the response and has data
        if "articles" not in data or not data["articles"]:
            msg = f"No news found for {country_code}."
            logger.warning(msg)
            return {
                "error": {"news": msg},
                "task_completed": {"news": False}
                   }

        # Extract the titles of the top 3 articles from the response
        titles = ", ".join([article["title"] for article in data["articles"][:3]])
        headlines = f"Headlines in {country_code.upper()}: {titles}"

        logger.info(f"Found headlines: {headlines}")

        # Return the news headlines as a dictionary
        return {"results": {"news": [headlines]},
                "task_completed": {"news": True}
}

    except Exception as e:
        # Handle any unexpected errors during the news retrieval process
        logger.exception("Unexpected error while fetching news")
        return {
            "error": {"news": f"Error fetching news: {str(e)}"},
            "task_completed": {"news" : False}
            }

### Prototipado del Nodo  de gestion de errores y su Prompt


In [17]:
test_inputs = [
    "¿Cuáles son las noticias hoy en UK?",
    "Dame los titulares más recientes de Estados Unidos.",
    "Noticias de Irak por favor",
    "¿Qué ha pasado últimamente en Argentina?",
    "¿Qué está sonando en las noticias?",
    "Me gustaría saber lo que pasa en Colombia y también cómo está el clima allá."
]

for input_text in test_inputs:
    state = {
                "messages": [HumanMessage(content=input_text)],
                "tasks_to_do": {},
                "results": {},
                "error": {},
                "order_task": None,
                "ready_to_aggregate": False,
            }

    output = get_news(state)

    print(f"Entrada: '{input_text}'")
    print(f"Salida: '{output}'")



Entrada: '¿Cuáles son las noticias hoy en UK?'
Salida: '{'error': {'news': 'No news found for uk.'}, 'task_completed': {'news': False}}'
Entrada: 'Dame los titulares más recientes de Estados Unidos.'
Salida: '{'results': {'news': ['Headlines in US: Stock Market Today: Dow Jumps 800 Points; Gold Hits $3,500 on Tariff, Fed Worries — Live Updates - WSJ, Harvard sues the Trump administration in escalating confrontation - The Washington Post, Google Fi is launching a $35 / month unlimited plan - The Verge']}, 'task_completed': {'news': True}}'
Entrada: 'Noticias de Irak por favor'
Salida: '{'error': {'news': 'No news found for iq.'}, 'task_completed': {'news': False}}'
Entrada: '¿Qué ha pasado últimamente en Argentina?'
Salida: '{'error': {'news': 'No news found for ar.'}, 'task_completed': {'news': False}}'
Entrada: '¿Qué está sonando en las noticias?'
Salida: '{'error': {'news': 'No news found for None.'}, 'task_completed': {'news': False}}'
Entrada: 'Me gustaría saber lo que pasa en Colo

In [18]:
# %%writefile nodes/error_handler.py

import logging
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from core.agent_state import AgentState  # Adjust if needed
from langchain_openai import ChatOpenAI

# ----- Configurar logging -----
from utils.logging import setup_logging

# Initialize logger using the setup_logging function
logger = setup_logging()

# ----- Cargar variables de entorno -----
load_dotenv(dotenv_path='env')

# Global LLM instance
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# ----- Error Interpretation Prompt -----
error_handler_template = PromptTemplate(
    input_variables=["error", "original_text"],
    template="""
Eres un asistente experto en interpretar errores de sistemas que consultan datos sobre clima, noticias y divisas.

Mensaje original del usuario:
"{original_text}"

Mensaje de error del sistema:
"{error}"

1. Si hay nombres de ciudades, países o monedas abreviados (como 'UK', 'US', 'EUR'), proporciónalos en su forma completa y clara.
2. Genera una explicación amigable del error para el usuario.
3. Sugiere una alternativa. Por ejemplo, si no se puede obtener el clima, sugiere obtener noticias o divisas, y viceversa.
reiterando que vuleva a hacer la peticion con la recomenacion asociada
Devuelve solo el texto final para el usuario, no incluyas explicaciones adicionales ni estructuras.
"""
)

# ----- Error Handler Node -----
def error_handler(state: AgentState) -> AgentState:
    """
    Uses LLM to transform technical error messages into user-friendly suggestions.

    Parameters:
        state (AgentState): The shared graph state including error and last message.

    Returns:
        dict: {"results": {...}, "task_completed": {...}, "error": {...}}
    """

    # Añadir trazabilidad del nodo
    state.setdefault("history", []).append("task_error")
    try:
        user_input = state["messages"][-1].content if state.get("messages") else ""

        errores = state.get("error", {})
        nodo_source = next(iter(errores), "desconocido")  # Tomamos la primera clave
        raw_error = errores.get(nodo_source, "Error no especificado")

        logger.info(f"Procesando error desde el nodo '{nodo_source}': {raw_error}")

        prompt = error_handler_template.format(
            error=raw_error,
            original_text=user_input
        )
        logger.debug(f"Prompt generado para el LLM:\n{prompt}")
        response = llm.invoke([HumanMessage(content=prompt)])
        friendly_message = response.content.strip()

        # Remover la clave procesada
        errores.pop(nodo_source, None)

        logger.info(f"Mensaje amigable generado: {friendly_message}")
        return {
            "results": {nodo_source: [friendly_message]},
            "task_completed": {nodo_source: True},
            "error": { "error": errores } # Retornamos el dict sin la clave ya procesada
        }

    except Exception as e:
        logger.exception("Error en el manejador de errores")
        fallback_message = f"No se pudo procesar el error automáticamente. Detalles: {str(e)}"
        return {
            "results": {"error": [fallback_message]},
            "task_completed": {'error': True}
        }


In [19]:


state = {
            "messages": [HumanMessage(content="¿Qué clima hay en UK?")],
            "tasks_to_do": {},
            "results": {},
            "error": {
            "weather" : "Error al obtener el estado del clima:  La API esta fuera de servicio por el momento}"
            },
            "order_task": None,
            "ready_to_aggregate": False,
        }

print(error_handler(state)['results'])

{'weather': ['Lo siento, hubo un error al obtener el estado del clima en el Reino Unido. La API está fuera de servicio por el momento. Te recomendaría consultar las noticias o divisas en su lugar. Por favor, vuelve a hacer la petición con la recomendación asociada.']}


In [20]:
state

{'messages': [HumanMessage(content='¿Qué clima hay en UK?', additional_kwargs={}, response_metadata={})],
 'tasks_to_do': {},
 'results': {},
 'error': {},
 'order_task': None,
 'ready_to_aggregate': False,
 'history': ['task_error']}

### Prototipado del Nodo order


In [21]:
# %%writefile nodes/order_tasks.py

import json
import logging
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from core.agent_state import AgentState
from langchain_openai import ChatOpenAI

# ----- Configurar logging -----
from utils.logging import setup_logging

# Initialize logger using the setup_logging function
logger = setup_logging()

# ----- Cargar variables de entorno -----
load_dotenv(dotenv_path='env')

# Global LLM instance
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# ----- Task Ordering Prompt -----
order_tasks_template = PromptTemplate(
    input_variables=["tasks", "user_input"],
    template="""
Eres un asistente experto en coordinar tareas de un sistema que puede obtener información sobre clima, noticias y divisas.

El usuario ha solicitado información sobre las siguientes tareas: {tasks}.

Consulta original del usuario:
"{user_input}"

Analiza la intención del usuario y determina el orden más lógico y útil en el que se deben ejecutar estas tareas. 
Devuelve únicamente un objeto JSON donde cada clave sea el nombre de la tarea y su valor sea su posición en el orden en el que aparece en el texto ,
si no aparece alguna tarea omitela de la respuesta
por ejemplo: {{"weather": 1, "exchange": 2, "noticias": 3}}

No incluyas ningún otro texto ni explicaciones.
"""
)

# ----- Task Ordering Node -----
def order_tasks(state: AgentState) -> AgentState:
    """
    Orders tasks based on the user's input and their intent, helping to prioritize actions.

    Parameters:
        state (AgentState): The shared state that includes tasks and the user's query.

    Returns:
        dict: {"results": {"order": ...}, "task_completed": {"order": True}}
    """
    # Añadir trazabilidad del nodo
    state.setdefault("history", []).append("task_order")
    tasks = {k: v for k, v in state.get("tasks", {}).items() if v}
    user_input = state["messages"][-1].content if state.get("messages") else ""

    logger.info(f"Tareas detectadas: {list(tasks.keys())}")
    logger.debug(f"Consulta del usuario: {user_input}")

    try:
        prompt = order_tasks_template.format(
            tasks=", ".join(tasks.keys()),
            user_input=user_input
        )
        logger.debug(f"Prompt generado para el LLM:\n{prompt}")

        response = llm.invoke([HumanMessage(content=prompt)])
        ordered_dict = json.loads(response.content.strip())

        logger.info(f"Orden propuesto por LLM: {ordered_dict}")

        return {
            "order_task":  ordered_dict,
            "task_completed": {'order':True}
        }

    except Exception as e:
        error_msg = f"Error al ordenar tareas: {str(e)}"
        logger.exception(error_msg)
        return {
            "task_completed": {'order':False},
            "error": {"order": error_msg}
        }


In [22]:
from core.agent_state import AgentState  # Ajusta la importación según tu estructura

# Estado de ejemplo
state = {
    
    "messages": [HumanMessage(content="Quiero saber el clima y las noticias, pero en orden de importancia")],
    "tasks": {
        "weather": True,  # Tarea activa
        "exchange": True,  # Tarea activa
        "news": True  # Tarea activa
    },
    "error": {},
}



In [23]:

# Llama la función order_tasks
new_state = order_tasks(state)

# Imprime el nuevo estado para verificar la respuesta
print(new_state)  # Aquí deberías ver el diccionario con el orden propuesto


{'order_task': {'weather': 1, 'news': 2}, 'task_completed': {'order': True}}


### Prototipado del Nodo integrador


In [24]:
# %%writefile nodes/aggregator_tasks.py

import logging
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from core.agent_state import AgentState  # Ajustar según sea necesario

# ----- Configurar logging -----
from utils.logging import setup_logging

# Initialize logger using the setup_logging function
logger = setup_logging()

# ----- Cargar variables de entorno -----
load_dotenv(dotenv_path='env')

# Instancia global de LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Plantilla para "enchulamiento"
enchulador_template = PromptTemplate(
    input_variables=["mensaje"],
    template="""
Enchula el siguiente mensaje para hacerlo amigable para el usuario:

{mensaje}
"""
)

def aggregator(state: AgentState) -> AgentState:
    """
    Reformula resultados exitosos con el LLM y agrega errores directamente.
    Devuelve mensajes listos para mostrar al usuario.

    Args:
        state (dict): Contiene 'order_task', 'results', 'error'.

    Returns:
        dict: {"results": {"aggregator": [messages]}, "task_completed":  {}}
    """
    # Añadir trazabilidad del nodo
    state.setdefault("history", []).append("task_aggregator")
    processed_messages = []

    logger.info("Iniciando agregación de tareas...")

    # Ordenar las tareas según el orden dado en el diccionario order_task
    # Ordenamos el diccionario order_task por los valores (el orden de las tareas)
    sorted_order = sorted(state.get("order_task", {}).items(), key=lambda item: item[1])
    logger.info("Orden detectado  %s", sorted_order)

    # Iterar en el orden correcto
    for task, order in sorted_order:
        result = state.get("results", {}).get(task)
        error = state.get("error", {}).get(task)

        logger.debug(f"Tarea: {task} | Orden: {order} | Resultado: {result} | Error: {error}")


        if result:
            mensaje_bruto = f"{task.capitalize()}: {result}"
            try:
                prompt_text = enchulador_template.format(mensaje=mensaje_bruto)
                prompt = HumanMessage(content=prompt_text)
                response = llm.invoke([prompt])
                friendly_text = response.content.strip()
                logger.info(f"Mensaje procesado para '{task}': {friendly_text}")
                processed_messages.append(friendly_text)
            except Exception as e:
                logger.exception(f"No se pudo reformular el mensaje para '{task}': {str(e)}")
                processed_messages.append(mensaje_bruto)

    # Retornar solo el formato correcto, sin modificar el state
    return {
        "results": {"aggregator": processed_messages},
        "task_completed": {'aggregator':True}
    }


In [25]:
# Simulación de estado con resultados y errores
mock_state = {
    "order_task": {  # Usamos 'order_task' con el orden de las tareas
        "weather": 1,
        "exchange": 2,
        "news": 3
    },
    "results": {
        "weather": "En CDMX hay 22°C y cielo parcialmente nublado.",
        "exchange": "No se pudo obtener el tipo de cambio. Verifica la moneda solicitada.",
        "news": "Elecciones en España dominan los titulares europeos."
    },
    "error": {

    }
}



# Ejecutamos la función
updated_state = aggregator(mock_state)



In [26]:
updated_state

{'results': {'aggregator': ['¡Hola! Te informo que en la Ciudad de México tenemos una temperatura de 22°C y el cielo se encuentra parcialmente nublado. ¡Que tengas un excelente día! 🌤️🌡️',
   '¡Hola! Parece que no pudimos obtener el tipo de cambio para la moneda que solicitaste. Te recomendamos verificar la moneda que ingresaste. ¡Gracias por tu comprensión!',
   '¡Hola! Te contamos que las elecciones en España están acaparando la atención de los titulares europeos. ¡No te pierdas las últimas noticias!']},
 'task_completed': {'aggregator': True}}